<a href="https://colab.research.google.com/github/br0ast/KRaE-lab/blob/main/RDFLib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
pip install rdflib

In [59]:
from rdflib import *

In [60]:
examplegraph = Graph() #create a graph

In [61]:
namespacestring = "https://www.example.org/" #create a string with a namespace
# like http://dbpedia.org/resource/ in dbpedia
example = Namespace(namespacestring) #create a Namespace with the Namespace constructor method

In [62]:
examplegraph.bind("ex", example) #add the prefix to the graph

In [63]:
examplegraph.add((example.Bruno, example.isTutorOf, example.DHDK)) # adding a triple to the graph, the add function takes a tuple with 3 elements 
#(subject, property, object) as an input, and adds the triple to the graph

<Graph identifier=N9aa9dc2e4bfb45c0a363ae157364b039 (<class 'rdflib.graph.Graph'>)>

In [64]:
examplegraph.add((example.Bruno, example.hasRole, example.Tutor))

<Graph identifier=N9aa9dc2e4bfb45c0a363ae157364b039 (<class 'rdflib.graph.Graph'>)>

In [65]:
examplegraph.add((example.Bruno, example.hasRole, example.PhDStudent))

<Graph identifier=N9aa9dc2e4bfb45c0a363ae157364b039 (<class 'rdflib.graph.Graph'>)>

In [66]:
examplegraph.add((example.Bruno, RDFS.label, Literal("Bruno"))) #Literal is used to create a literal to add to the graph, you can't add strings directly

<Graph identifier=N9aa9dc2e4bfb45c0a363ae157364b039 (<class 'rdflib.graph.Graph'>)>

In [67]:
examplegraph.add((example.Bruno, example.livesIn, URIRef("http://dbpedia.org/resource/Amsterdam"))) #URIRef is used when the URI of the element you are
# adding is not bound to the graph or when you iterate to get multiple triples at the same time

<Graph identifier=N9aa9dc2e4bfb45c0a363ae157364b039 (<class 'rdflib.graph.Graph'>)>

In [68]:
print(examplegraph.serialize(format="ttl"))

@prefix ex: <https://www.example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Bruno rdfs:label "Bruno" ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn <http://dbpedia.org/resource/Amsterdam> .




In [69]:
dbr = Namespace("http://dbpedia.org/resource/")

In [70]:
examplegraph.bind("dbr", dbr)

In [71]:
print(examplegraph.serialize(format="ttl"))

@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <https://www.example.org/> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .

ex:Bruno rdfs:label "Bruno" ;
    ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Amsterdam .




In [72]:
automaticallyconvertedgraph = Graph()
automaticallyconvertedgraph.bind("ex", example)
automaticallyconvertedgraph.bind("dbr", dbr)

In [73]:
startingdata = {"ex:Bruno":{"literals":{"ex:name":"Bruno", "ex:age":26}, "uris":{"ex:hasRole":["ex:PhDStudent", "ex:Tutor"], "ex:isTutorOf":"ex:DHDK", "ex:livesIn":"dbr:Amsterdam"}},
                "ex:Valentina":{"literals":{"ex:name":"Valentina", "ex:age":26}, "uris":{"ex:hasRole":["ex:PhDStudent", "ex:Tutor"], "ex:isTutorOf":"ex:DHDK", "ex:livesIn":"dbr:Bologna"}}}

In [74]:
dbpediauri = "http://dbpedia.org/resource/"
for person in startingdata:
  if "ex:" in person:
    personuri = URIRef(person.replace("ex:", namespacestring))
    for literal in startingdata[person]["literals"]:
      if type(startingdata[person]["literals"][literal]) == str:
        lit = startingdata[person]["literals"][literal]
        automaticallyconvertedgraph.add((personuri, URIRef(literal.replace("ex:", namespacestring)), Literal(lit)))
    for uri in startingdata[person]["uris"]:
      if type(startingdata[person]["uris"][uri]) == str:
        uristring = startingdata[person]["uris"][uri]
        if "ex:" in uristring:
          uristring = uristring.replace("ex:", namespacestring)
        if "dbr:" in uristring:
          uristring = uristring.replace("dbr:", dbpediauri)
        automaticallyconvertedgraph.add((personuri, URIRef(uri.replace("ex:", namespacestring)), URIRef(uristring)))
      else:
        for multiobject in startingdata[person]["uris"][uri]:
          uristring = multiobject.replace("ex:", namespacestring)
          automaticallyconvertedgraph.add((personuri, URIRef(uri.replace("ex:", namespacestring)), URIRef(uristring)))


In [75]:
print(automaticallyconvertedgraph.serialize(format="ttl"))

@prefix dbr: <http://dbpedia.org/resource/> .
@prefix ex: <https://www.example.org/> .

ex:Bruno ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Amsterdam ;
    ex:name "Bruno" .

ex:Valentina ex:hasRole ex:PhDStudent,
        ex:Tutor ;
    ex:isTutorOf ex:DHDK ;
    ex:livesIn dbr:Bologna ;
    ex:name "Valentina" .




In [76]:
for s,p,o in automaticallyconvertedgraph.triples((None, None, None)): #iterates over triples without specifing any 
  print(s,p,o)

https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Bologna
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/isTutorOf https://www.example.org/DHDK
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Bruno https://www.example.org/livesIn http://dbpedia.org/resource/Amsterdam
https://www.example.org/Bruno https://www.example.org/name Bruno
https://www.example.org/Bruno https://www.example.org/isTutorOf https://www.example.org/DHDK
https://www.example.org/Valentina https://www.example.org/name Valentina


In [77]:
for s,p,o in automaticallyconvertedgraph.triples((None, example.hasRole, None)): #iterates over triples that have a specific property
  print(s,p,o)

https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Bruno https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/Tutor


In [78]:
for s,p,o in automaticallyconvertedgraph.triples((example.Valentina, None, None)):
  print(s,p,o) #iterates over triples that have a specific subject

https://www.example.org/Valentina https://www.example.org/name Valentina
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/PhDStudent
https://www.example.org/Valentina https://www.example.org/hasRole https://www.example.org/Tutor
https://www.example.org/Valentina https://www.example.org/isTutorOf https://www.example.org/DHDK
https://www.example.org/Valentina https://www.example.org/livesIn http://dbpedia.org/resource/Bologna


In [80]:
querytext = '''
prefix ex: <https://www.example.org/>
select distinct ?people where { ?people ex:hasRole ?role}
'''
result = automaticallyconvertedgraph.query(querytext) #you can query them too
for el in result:
  print(el["people"])

https://www.example.org/Bruno
https://www.example.org/Valentina


In [81]:
pip install SPARQLWrapper

In [86]:
from SPARQLWrapper import SPARQLWrapper, JSON

endpoint_url = "https://dbpedia.org/sparql/"

query = """
prefix dbr: <http://dbpedia.org/resource/> 
SELECT ?p ?o
WHERE {dbr:Amsterdam ?p ?o}
LIMIT 25
"""

sparql = SPARQLWrapper(endpoint_url)
sparql.setQuery(query)
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result["p"]["value"], result["o"]["value"])


http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2002/07/owl#Thing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/Place
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/Location
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/ontology/PopulatedPlace
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.w3.org/2003/01/geo/wgs84_pos#SpatialThing
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://schema.org/City
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://schema.org/Place
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.wikidata.org/entity/Q486972
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://www.wikidata.org/entity/Q515
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/class/yago/WikicatPopulatedPlacesEstablishedInThe13thCentury
http://www.w3.org/1999/02/22-rdf-syntax-ns#type http://dbpedia.org/class/yago/WikicatPopulatedPlacesInNorthHolland
